<a href="https://colab.research.google.com/github/Taramas73/DS-final-project/blob/irusha/Hybrid_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Hybrid Model: EfficientNet Encoder + U-Net Decoder
Here's how you can implement this architecture:

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, BatchNormalization, Activation, Concatenate

def build_efficient_unet(input_shape=(256, 256, 3), n_classes=1):
    # Input layer
    inputs = Input(shape=input_shape)

    # EfficientNet encoder (pre-trained)
    base_model = EfficientNetB0(include_top=False, weights='imagenet', input_tensor=inputs)

    # Get specific activation layers from EfficientNet
    # These will be used for skip connections
    s1 = base_model.get_layer('block2a_expand_activation').output  # 128x128
    s2 = base_model.get_layer('block3a_expand_activation').output  # 64x64
    s3 = base_model.get_layer('block4a_expand_activation').output  # 32x32
    s4 = base_model.get_layer('block6a_expand_activation').output  # 16x16

    # Bridge
    b1 = base_model.get_layer('top_activation').output  # 8x8

    # Decoder path
    # Upsampling block 1
    d1 = Conv2DTranspose(256, (3, 3), strides=(2, 2), padding='same')(b1)
    d1 = BatchNormalization()(d1)
    d1 = Activation('relu')(d1)
    d1 = Concatenate()([d1, s4])
    d1 = Conv2D(256, (3, 3), padding='same')(d1)
    d1 = BatchNormalization()(d1)
    d1 = Activation('relu')(d1)

    # Upsampling block 2
    d2 = Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same')(d1)
    d2 = BatchNormalization()(d2)
    d2 = Activation('relu')(d2)
    d2 = Concatenate()([d2, s3])
    d2 = Conv2D(128, (3, 3), padding='same')(d2)
    d2 = BatchNormalization()(d2)
    d2 = Activation('relu')(d2)

    # Upsampling block 3
    d3 = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same')(d2)
    d3 = BatchNormalization()(d3)
    d3 = Activation('relu')(d3)
    d3 = Concatenate()([d3, s2])
    d3 = Conv2D(64, (3, 3), padding='same')(d3)
    d3 = BatchNormalization()(d3)
    d3 = Activation('relu')(d3)

    # Upsampling block 4
    d4 = Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same')(d3)
    d4 = BatchNormalization()(d4)
    d4 = Activation('relu')(d4)
    d4 = Concatenate()([d4, s1])
    d4 = Conv2D(32, (3, 3), padding='same')(d4)
    d4 = BatchNormalization()(d4)
    d4 = Activation('relu')(d4)

    # Final upsampling
    d5 = Conv2DTranspose(16, (3, 3), strides=(2, 2), padding='same')(d4)
    d5 = BatchNormalization()(d5)
    d5 = Activation('relu')(d5)

    # Output layer
    if n_classes == 1:  # Binary segmentation
        outputs = Conv2D(1, (1, 1), activation='sigmoid')(d5)
    else:  # Multi-class segmentation
        outputs = Conv2D(n_classes, (1, 1), activation='softmax')(d5)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    # Freeze the encoder weights initially
    for layer in base_model.layers:
        layer.trainable = False

    return model